<a href="https://colab.research.google.com/github/hnguyen8/get_video_statistics_from_Good_Mythical_Morning/blob/main/get_data_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# libraries
import requests  # can make API calls, works with all APIs
import pandas as pd
import time

In [38]:
# Keys
API_KEYS = 'xxxxx'
CHANNEL_ID = 'xxxxx'

In [ ]:
# collect video counts
url_channel_stat = 'https://www.googleapis.com/youtube/v3/channels?part=statistics&id='+CHANNEL_ID+'&key='+API_KEYS
response_channel_stats = requests.get(url_channel_stat).json()

In [58]:
video_count = response_channel_stats['items'][0]['statistics']['videoCount']
video_count

'3031'

In [59]:
def get_video_details(video_id):
  # collect view, like, favorite, comment counts
  # Reference: https://developers.google.com/youtube/v3/docs/videos
  url_video_stat = 'https://www.googleapis.com/youtube/v3/videos?id='+video_id+'&part=statistics&key='+API_KEYS
  response_video_stats = requests.get(url_video_stat).json()
  
  view_count = response_video_stats['items'][0]['statistics']['viewCount']
  like_count = response_video_stats['items'][0]['statistics']['likeCount']
  favorite_count = response_video_stats['items'][0]['statistics']['favoriteCount']
  comment_count =response_video_stats['items'][0]['statistics']['commentCount']

  return view_count, like_count, favorite_count, comment_count

In [60]:
def get_videos(df):
  # make API call to collect video ids and snippets
  # Reference: https://developers.google.com/youtube/v3/docs/search
  pageToken = ''
  url = 'https://www.googleapis.com/youtube/v3/search?key='+API_KEYS+'&channelId='+CHANNEL_ID+'&part=snippet,id&order=date&maxResults=3032'+pageToken  # HTTP request

  response = requests.get(url).json()
  time.sleep(1) # wait 1 sec - let API call finishes

  for video in response['items']:
    if video['id']['kind'] == 'youtube#video': # allow Youtube videos only 
      video_id = video['id']['videoId']
      video_title = video['snippet']['title']
      upload_date = video['snippet']['publishTime']
      upload_date = str(upload_date).split('T')[0]   # keep only date

       # call function to collect view, like, favorite, comment counts
      view_count, like_count, favorite_count, comment_count = get_video_details(video_id)

      # save into pandas dataframe 
      df = df.append({ 'video_id':video_id , 'video_title':video_title, 'upload_date':upload_date, 'view_count': view_count, 'like_count': like_count,
                      'favorite_count': favorite_count, 'comment_count': comment_count }, ignore_index= True )
  return df

In [61]:
# build blank dataframe
df = pd.DataFrame(columns = ['video_id', 'video_title', 'upload_date', 'view_count', 'like_count', 'favorite_count', 'comment_count'])

##
df = get_videos(df) 

In [62]:
df.head()

,video_id,video_title,upload_date,view_count,like_count,favorite_count,comment_count
0,w42iXLlQK1Q,What Snacks Go Stale The Fastest?,2022-10-21,231166,13004,0,605
1,BH8_FkQ3ct4,Who Can Start A Fire Faster? (Challenge),2022-10-20,632972,25607,0,1346
2,c4EUhTEojxo,Exotic Fruits Taste Test,2022-10-19,933478,31011,0,1848
3,LZZbg9AVtzs,Testing Discontinued Products,2022-10-18,714025,24574,0,1032
4,g_1vhq8nMA4,Frozen vs. Fast vs. Fancy Food Taste Test,2022-10-17,1224218,39625,0,1721


In [64]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [65]:
# save to csv in Drive
df.to_csv('/content/gdrive/My Drive/DataScience/good_mythical_morning_videos.csv')